In [1]:
# Libraries and functions
# Importing the necessary libraries
import pandas as pd
import numpy as np

# Function to calculate proportions
def calculate_proportions(df1, col1, col2, col3):
    df_new[col1] = df1[col2] / df1[col3]
    return df_new

# Function to normalize the values of a column
def normalize(df, col1, col2):
    min_density = df[col1].min()
    max_density = df[col1].max()
    df_new[col2] =(df[col1] - min_density) / (max_density - min_density)
    return df_new

In [2]:
# Load the control variable data
df = pd.read_csv("Data/data.csv", index_col=0)

# Load the livability data (from CBS)
df_buurt = pd.read_csv("Data/Leefbaarometer-scores buurten 2002-2022.csv")

# Select the first four columns: the code, the name, the year, and the livability score
df_buurt = df_buurt.iloc[:, :4]

# Filter for the years 2020 and 2022
df_buurt = df_buurt[(df_buurt['jaar'] == 2020) | (df_buurt['jaar'] == 2022)]

# Remove the first two characters from 'bu_code': BU001400 to 001400
df_buurt["bu_code"] = df_buurt["bu_code"].apply(lambda x: x[2:])

# Rename the 'bu_naam' column to 'name'
df_buurt.rename(columns={'bu_naam': 'name'}, inplace=True)

# Initialize an empty list to store the averaged rows
dfall = []

# Iterate over the dataframe in steps of 2 to pair rows
for i in range(0, len(df_buurt) - 1, 2):
    row1 = df_buurt.iloc[i]
    row2 = df_buurt.iloc[i + 1]
    
    # Calculate the average of the last column values
    average_value = (row1[-1] + row2[-1]) / 2
    
    # Create a new row with the max value of the first column, year as 2021, and the average value
    new_row = [max(row1[0], row2[0]), 2021, average_value]
    
    # Append the new row to the list
    dfall.append(new_row)

# Convert the list of new rows to a DataFrame
df_all = pd.DataFrame(dfall, columns=["code", "year", "livability_score"])

# Convert the 'code' column to integer type
df_all["code"] = df_all["code"].astype(int)

# Merge the control variable dataframe with the livability dataframe on the 'code' column
df=pd.merge(df, df_all, left_on="gwb_code_10",right_on="code", how='inner')

C:\Users\I6240624\AppData\Local\Temp\ipykernel_20312\3954079681.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  average_value = (row1[-1] + row2[-1]) / 2
C:\Users\I6240624\AppData\Local\Temp\ipykernel_20312\3954079681.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_row = [max(row1[0], row2[0]), 2021, average_value]


In [3]:
# Column renaming and selection for better readability
new_column_names = {
    "gwb_code_10": "postal_code",
    "DistanceToGPPractice_5": "Distance GP", 
    "Within1Km_6": "Within 1 km GP", 
    "Within3Km_7": "Within 3 km GP", 
    "Within5Km_8": "Within 5 km GP",
    "DistanceToHospital_11": "Distance Hospital",
    "Within5Km_12": "Within 5 km Hospital",
    "Within10Km_13": "Within 10 km Hospital", 
    "Within20Km_14": "Within 20 km Hospital",
    "DistanceToLargeSupermarket_24": "Distance Supermarket", 
    "Within1Km_25": "Within 1 km Supermarket", 
    "Within3Km_26": "Within 3 km Supermarket", 
    "Within5Km_27": "Within 5 km Supermarket",
    "DistanceToDepartmentStore_32": "Distance Department Store",
    "Within5Km_33": "Within 5 km Department Store",
    "Within10Km_34": "Within 10 km Department Store",
    "Within20Km_35": "Within 20 km Department Store",
    "DistanceToRestaurant_44": "Distance Restaurant", 
    "Within1Km_45": "Within 1 km Restaurant", 
    "Within3Km_46": "Within 3 km Restaurant", 
    "Within5Km_47": "Within 5 km Restaurant",
    "DistanceToDaycareCentres_52": "Distance Day Care", 
    "Within1Km_53": "Within 1 km Day Care", 
    "Within3Km_54": "Within 3 km Day Care", 
    "Within5Km_55": "Within 5 km Day Care",
    "DistanceToSchool_60": "Distance School", 
    "Within1Km_61": "Within 1 km School", 
    "Within3Km_62": "Within 3 km School", 
    "Within5Km_63": "Within 5 km School"
}

# Renaming columns and selecting specified columns
dist_df = df.rename(columns=new_column_names)[list(new_column_names.values())]

# Exporting the DataFrame to a CSV file for regression analysis
dist_df.to_csv("Data/distances.csv", index=False)

In [4]:
# Create a new DataFrame
df_new = pd.DataFrame()

# Copy 'postal_code' column from existing DataFrame 'df' to the new DataFrame 'df_new'
df_new["postal_code"] = df["gwb_code_10"]

In [5]:
# Calculate the weighted average age for each row
df_new['Weighted_Average_Age'] = (
                                         df['Numbers.of.people.from.0.14.years.old'] * 7 +
                                         df['Number.of.people.from.15.24.years.old'] * 20 +
                                         df['Number.of.people.from.25.44.years.old'] * 35 +
                                         df['Number.of.people.from.45.64.years.old'] * 55 +
                                         df['Number.of.people....65.years.old'] * 70
                                 ) / df['Number.of.inhabitants']


# Calculate the variance for each row
df_new['Variance_Age'] = (
                                 (df['Numbers.of.people.from.0.14.years.old'] * (
                                             7 - df_new['Weighted_Average_Age']) ** 2) +
                                 (df['Number.of.people.from.15.24.years.old'] * (
                                             20 - df_new['Weighted_Average_Age']) ** 2) +
                                 (df['Number.of.people.from.25.44.years.old'] * (
                                             35 - df_new['Weighted_Average_Age']) ** 2) +
                                 (df['Number.of.people.from.45.64.years.old'] * (
                                             55 - df_new['Weighted_Average_Age']) ** 2) +
                                 (df['Number.of.people....65.years.old'] * (
                                             70 - df_new['Weighted_Average_Age']) ** 2)
                         ) / df['Number.of.inhabitants']

# Standard deviation is the square root of the variance
df_new['Standard_Deviation_Age'] = np.sqrt(df_new['Variance_Age'])
df_new=df_new.drop(columns="Variance_Age")

In [6]:
# Calculate proportions for age groups
calculate_proportions(df, 'prop_0-14', 'Numbers.of.people.from.0.14.years.old', 'Number.of.inhabitants')


calculate_proportions(df, 'prop_15-24', 'Number.of.people.from.15.24.years.old', 'Number.of.inhabitants')


calculate_proportions(df, 'prop_25-44', 'Number.of.people.from.25.44.years.old', 'Number.of.inhabitants')


calculate_proportions(df, 'prop_45-64', 'Number.of.people.from.45.64.years.old', 'Number.of.inhabitants')


calculate_proportions(df, 'prop_65+', 'Number.of.people....65.years.old', 'Number.of.inhabitants')


# Calculate proportion for education level
df = df.drop(columns=['n_highschool_educated'])
calculate_proportions(df, 'prop_university', 'n_highly_educated', 'Number.of.inhabitants')
df = df.drop(columns=['n_highly_educated'])

# Calculate proportion for income recipients
calculate_proportions(df, 'prop_recipients', 'n_income_recipients', 'Number.of.inhabitants')
df_new["worker"] = 1 - df_new['prop_recipients']
df_new = df_new.drop(columns=['prop_recipients'])
df = df.drop(columns=['n_income_recipients'])

# Calculate proportion for women
calculate_proportions(df, 'prop_women', 'Number.of.women', 'Number.of.inhabitants')
df = df.drop(columns=['Number.of.women'])

In [7]:
# Normalize population density
normalize(df, 'Population.density', 'pop_density')
df = df.drop(columns=['Population.density'])

# Include livability score
df_new["livability_score"]=df["livability_score"]*2

# Calculate and normalize total aid
df["Aid"] = df["n_pp_w_AO"] + df["n_pp_w_WW"]
normalize(df, "Aid", "Aid")
df = df.drop(columns=['Aid'])

# Normalize average income per recipient
normalize(df, "avg_income_per_recipient", "Income")

# Convert percentage of single-family housing to decimal and assign to new DataFrame
df_new["single_family_housing"] = df["percent.of.single.family.housing"] / 100
df = df.drop(columns=['percent.of.single.family.housing'])

# Drop rows with missing values in the new DataFrame
df_new.dropna(inplace=True)

In [8]:
df_new

,postal_code,Weighted_Average_Age,Standard_Deviation_Age,prop_0-14,prop_15-24,prop_25-44,prop_45-64,prop_65+,prop_university,worker,prop_women,pop_density,livability_score,Aid,Income,single_family_housing
0,340101,47.443038,21.552028,0.098101,0.094937,0.253165,0.199367,0.357595,0.170886,0.177215,0.534810,0.085318,8.226781,0.058480,0.119906,0.03
1,340102,40.202797,21.803908,0.180070,0.101399,0.283217,0.239510,0.197552,0.171329,0.195804,0.517483,0.158171,7.931987,0.140351,0.103448,0.65
2,340201,40.317560,19.234362,0.116719,0.096740,0.401682,0.223975,0.159832,0.319664,0.263933,0.481598,0.157137,8.070317,0.198830,0.170063,0.01
3,340202,39.320709,20.525905,0.150527,0.157718,0.244487,0.316874,0.130393,0.246405,0.194631,0.512464,0.137107,8.062306,0.345029,0.163009,0.68
4,340203,36.697898,19.934602,0.181261,0.155867,0.268827,0.316112,0.078809,0.182137,0.229422,0.496497,0.245857,7.932075,0.187135,0.137931,0.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957,3639700,38.842905,20.597952,0.160473,0.136824,0.295608,0.265203,0.143581,0.165541,0.222973,0.498311,0.259273,7.732136,0.122807,0.090909,0.00
958,3639701,35.979042,19.471641,0.182635,0.140719,0.314371,0.284431,0.074850,0.227545,0.221557,0.517964,0.266512,7.500271,0.081871,0.087774,0.00
959,3639702,43.046154,21.198026,0.138462,0.115385,0.220000,0.324615,0.203077,0.295385,0.169231,0.492308,0.137488,7.715644,0.093567,0.174765,1.00
960,3639703,41.746647,20.991594,0.150522,0.095380,0.271237,0.293592,0.187779,0.211624,0.195231,0.511177,0.126140,7.862723,0.128655,0.123041,0.56


In [9]:
# Save the new DataFrame with normalized variables to a CSV file
df_new.to_csv("Data/normalized_variables.csv")